<a href="https://colab.research.google.com/github/Chthanh/Recommender-System/blob/main/Item-based%20Collaborative%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import pandas as pd
import numpy as np

In [ ]:
# Load the dataset
data = pd.read_excel('data_02.xlsx')
data.rename(columns={"User": "userId"}, inplace=True)
data.set_index('userId', inplace = True)
data.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,1: Toy Story (1995),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),260: Star Wars: Episode IV - A New Hope (1977),2028: Saving Private Ryan (1998),296: Pulp Fiction (1994),1259: Stand by Me (1986),2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
userId,,,,,,,,,,,,,,,,,,,,
755,2.0,5.0,2.0,NaN,4.0,4.0,1.0,2.0,NaN,3.0,2.0,NaN,5.0,2.0,5.0,4.0,2.0,5.0,NaN,NaN
5277,1.0,NaN,NaN,2.0,4.0,2.0,5.0,NaN,NaN,4.0,3.0,2.0,2.0,NaN,2.0,NaN,5.0,1.0,3.0,NaN
1577,NaN,NaN,NaN,5.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0
4388,2.0,3.0,NaN,NaN,NaN,1.0,NaN,3.0,4.0,NaN,NaN,4.0,NaN,3.0,5.0,NaN,5.0,1.0,1.0,2.0
1202,NaN,3.0,4.0,1.0,4.0,1.0,4.0,4.0,NaN,1.0,5.0,1.0,NaN,4.0,NaN,3.0,5.0,5.0,NaN,NaN


In [ ]:
def cosine(item_i, item_j, adjust):

  i_avg = np.nansum(item_i)/np.count_nonzero(~np.isnan(item_i)) if adjust == True else 0
  j_avg = np.nansum(item_j)/np.count_nonzero(~np.isnan(item_j)) if adjust == True else 0

  mutual_index = [u for u in item_i.index if not np.isnan(item_i[u]) and not np.isnan(item_j[u])]

  dividend = 0
  i_divisor = 0
  j_divisor = 0

  for index in mutual_index:
    i_nr = item_i[index] - i_avg
    j_nr = item_j[index] - j_avg
    dividend += i_nr * j_nr
    i_divisor += pow(i_nr, 2)
    j_divisor += pow(j_nr, 2)

  divisor = np.sqrt(i_divisor) * np.sqrt(j_divisor)

  if divisor != 0:
    return round(dividend / divisor, 3)

  return 0


def cosine_mx(df, adjust):
  m, n = df.shape
  corr_mx = np.identity(n)
  for i in range(n-1):
    for j in range(i+1,n):
      corr_mx[i,j] = corr_mx[j,i] = cosine(df.iloc[:, i], df.iloc[:, j], adjust)
  corr_mx = pd.DataFrame(corr_mx, index=df.columns, columns=df.columns)

  return corr_mx


In [ ]:
def item_based_rec(data, target_user, k=2, adjust = False):
  """Input: data là tập dữ liệu có số dòng là userID và số cột là itemID"""

  # print("Ma trận ban đầu")
  # display(data)
  # Bước 1: Mean-centering
  normalized_df = data.subtract(data.mean(axis=1), axis='rows')

  # Bước 2: Ma trận tương đồng giữa các items
  corr_mx = cosine_mx(normalized_df, adjust)
  # print("\n")
  # print("Ma trận tương đồng: ")
  # display(corr_mx)

  # Bước 3: Những item mà user mục tiêu đã đánh giá
  target_user_rated = data[data.index==target_user].dropna(axis=1) # items đã đánh giá
  target_user_rated_df = target_user_rated.T.reset_index().rename(columns={target_user:'rating', 'index':'item'}) # chuyển vị, reset lại index và đổi tên cột

  # print("\n")
  # print(f"Các item mà user mục tiêu đã đánh giá:")
  # display(target_user_rated_df)

  # Bước 4: Những item mà user mục tiêu chưa đánh giá
  target_user_unrated = data.drop(target_user_rated.columns, axis=1, errors='ignore') # drop những item đã đánh giá

  # print("\n")
  # print(f"Các item mà user mục tiêu chưa đánh giá: {target_user_unrated.columns.values}")

  # Khởi tạo dictionary to lưu những item chưa được đánh giá và số điểm dự đoán tương ứng
  rating_prediction ={}

  # Lặp qua từng item trong tập item chưa được đánh giá
  for picked_item in target_user_unrated:
    # Bước 5: Tính điểm tương đồng của picked_item với các item khác
    picked_item_sim_score = corr_mx[[picked_item]].reset_index().rename(columns={'index':'item', picked_item:'similarity_score'})
    # print("\n")
    # print(f"Độ tương đồng của {picked_item} với các item khác: ")
    # display(picked_item_sim_score)

      # Sắp xếp k items có điểm tương đồng theo thứ tự từ cao đến thấp
    target_user_rated_sim = pd.merge(left=target_user_rated_df,
                                     right=picked_item_sim_score,
                                     on='item',
                                     how='inner')\
                                     .sort_values('similarity_score', ascending=False)[:k]
    # print("\n")
    # print(f"Các item tương đồng nhất với item {picked_item}: ")
    # display(target_user_rated_sim)
    # print("\n")

    # Bước 6: Dự đoán số điểm mà user mục tiêu có thể đánh giá cho picked_item
    #predicted_rating = round((target_user_rated_sim['rating']*target_user_rated_sim['similarity_score']).sum()/target_user_rated_sim['similarity_score'].sum(),3)
    predicted_rating = round(np.average(target_user_rated_sim['rating'],
                                        weights=target_user_rated_sim['similarity_score']), 3)
    # Lưu predicted rating vào dictionary
    rating_prediction[picked_item] = predicted_rating

  # Bước 7: Chuyển dictionary thành dataframe và sắp xếp dữ liệu theo prediction_score
  pred_score = (pd.DataFrame(rating_prediction.items(), columns=['item', 'pred_score'])
                      .sort_values(by='pred_score', ascending=False))

    # Trả về dataframe gồm item mà target_user chưa đánh giá cùng với số điểm rating dự đoán
  return pred_score

**1. Recommend top 5 movies similarest to movies "Toy Story", according to the similarity decreasing**

---



In [ ]:
#  using cosine
normalized_df = data.subtract(data.mean(axis=1), axis='rows')
corr_mx = cosine_mx(normalized_df, adjust = False)
corr_mx["1: Toy Story (1995)"].sort_values(ascending = False)[1:6]

34: Babe (1995)                          0.827
356: Forrest Gump (1994)                 0.530
318: Shawshank Redemption, The (1994)    0.521
296: Pulp Fiction (1994)                 0.479
2028: Saving Private Ryan (1998)         0.445
Name: 1: Toy Story (1995), dtype: float64

In [ ]:
# Using Adjust_Cosine
normalized_df = data.subtract(data.mean(axis=1), axis='rows')
corr_mx = cosine_mx(normalized_df, adjust = True)
corr_mx["1: Toy Story (1995)"].sort_values(ascending = False)[1:6]

318: Shawshank Redemption, The (1994)    0.820
34: Babe (1995)                          0.759
296: Pulp Fiction (1994)                 0.622
356: Forrest Gump (1994)                 0.523
2028: Saving Private Ryan (1998)         0.378
Name: 1: Toy Story (1995), dtype: float64

**2. Recommned top 5 movies for user 5277 by predicting rating score that user 5277 might to give to the movies they haven't watched yet**




---



In [ ]:
# Using Cosine
cos_predscore = item_based_rec(data = data, target_user = 5277, k = 5)
cos_predscore

,item,pred_score
2,2028: Saving Private Ryan (1998),3.199
1,356: Forrest Gump (1994),3.004
0,1210: Star Wars: Episode VI - Return of the Je...,2.867
3,296: Pulp Fiction (1994),2.810
5,"2571: Matrix, The (1999)",2.685
4,541: Blade Runner (1982),2.530
6,34: Babe (1995),1.927


In [ ]:
# Using Adjust_cosine
adjustcos_predscore = item_based_rec(data = data, target_user = 5277, k = 5, adjust = True)
adjustcos_predscore

,item,pred_score
2,2028: Saving Private Ryan (1998),3.290
1,356: Forrest Gump (1994),2.900
4,541: Blade Runner (1982),2.870
0,1210: Star Wars: Episode VI - Return of the Je...,2.838
5,"2571: Matrix, The (1999)",2.826
3,296: Pulp Fiction (1994),2.280
6,34: Babe (1995),1.941


**3. Comment on the result when using two cosine functions.**

---



In [ ]:
pd.merge(cos_predscore, adjustcos_predscore, how='inner', on='item', suffixes=('_cosine', '_adjust_cosine')).round(3).head(10)

,item,pred_score_cosine,pred_score_adjust_cosine
0,2028: Saving Private Ryan (1998),3.199,3.290
1,356: Forrest Gump (1994),3.004,2.900
2,1210: Star Wars: Episode VI - Return of the Je...,2.867,2.838
3,296: Pulp Fiction (1994),2.810,2.280
4,"2571: Matrix, The (1999)",2.685,2.826
5,541: Blade Runner (1982),2.530,2.870
6,34: Babe (1995),1.927,1.941


2 phương pháp cho ra 2 kết quả khác nhau một chút, các item đầu cả 2 cách tính đều cho ra kết quả dự đoán đều giống nhau, ngoài ra một vài item như Pulp Fiction khi sử dụng Cosine thì cho kết quả 2.81 trong khi adjust_cosine cho ra kết quả 2.28 chênh lệch nhau nhiều.